# Lesson 4 : Mini Challenge

Use what you learned in Session 1 to claim testnet tokens from the Whipz sale contract which we saw in Session 3

You will need to copy and paste some of the code from there into this notebook (or somewhere else if you prefer)

Make sure you create a new wallet using Metamask since we will be handling the private key in code

There is one new thing we are learning this week: how to interact with a contract. You will see the code example below, you will need to integrate this with the code you write


# Some other useful info

Here is the contract address of the Whipz token sale contract

```python
whipz_contract = '0x856820ee9EE8137D1c651689Ab6D548f5ac81fA1'
```

Use this url to connect to the alchemy sepolia websocket provider

```python
wss = f'wss://eth-sepolia.g.alchemy.com/v2/{alchemy_apikey}'
```

Check that your chain id is 11155111, i.e. Ethereum sepolia chainid

```python
w3.eth.chain_id
```

For the etherscan sepolia API, which we use to load the abi, use this url

```python
abi_endpoint = f"https://api-sepolia.etherscan.io/api?module=contract&action=getabi&address={whipz_contract}&apikey={etherscan_apikey}"
```

### Interacting with a contract

In [8]:
#For testnet you should be able to use these values
#GAS_AMOUNT = 500000
#MAX_GAS_PRICE = '40'
#PRIORITY_FEE = '2.5'

def send_tx(wallet, pvtkey, contract, amount_eth, GAS_AMOUNT, MAX_GAS_PRICE, PRIORITY_FEE):
    
    # Check wallet balance    
    balance = w3.from_wei(w3.eth.get_balance(wallet), "ether")
    print(f"Current wallet balance is {balance}")
    
    # We always need to pass the nonce (current transaction count) to a new tx
    nonce = w3.eth.get_transaction_count(wallet)

    # amount_eth is the amount of eth we want to spend, eg 0.01. 
    # we need to convert this to wei
    payable_amt_wei = w3.to_wei(amount_eth, "ether")
    
    # Call your function
    #===================================================
    call_function = contract.functions.buyTokens(wallet).build_transaction(
        {"chainId": w3.eth.chain_id, 
         "from": wallet, 
         "nonce": nonce, 
         'gas': GAS_AMOUNT,
         'value' : payable_amt_wei,
         'maxFeePerGas': w3.to_wei(MAX_GAS_PRICE, 'gwei'),
         'maxPriorityFeePerGas': w3.to_wei(PRIORITY_FEE, 'gwei'),})
    #===================================================

    # Sign transaction
    signed_tx = w3.eth.account.sign_transaction(call_function, private_key=pvtkey)
    
    # Send transaction
    send_tx = w3.eth.send_raw_transaction(signed_tx.rawTransaction)
    return send_tx